<a href="https://colab.research.google.com/github/DibyaSadhukhan/Amazon_Review_Analysis/blob/main/Product_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#importing required packages
import pandas as pd


#Extracts the recommended products

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/DibyaSadhukhan/Amazon_Review_Analysis/main/Data/Products.csv')
data=data.drop(data.columns[0], axis=1)
data=data.nlargest(5,['number_of_reveiws','Rating'])
data.head(10)
data.to_csv('Top_products.csv')

#Extracts the products for the first page 

In [ ]:

df = pd.read_csv('/content/gdrive/MyDrive/Laptop_Reveiw/Data/Master_reveiw_2021-10-09.csv')
#changing the data type of rating column 
df = df.rename(columns={'number of reveiws': 'number_of_reveiws'})
df['Rating']=df['Rating'].apply(lambda x : float(x.split(' ')[0]))
df=df.drop(df.columns[[0,-1,-2,-3]], axis=1)
#removing duplicate products
#df.loc[df['Product_code']=='B08SML9W2R']
df=df.drop_duplicates(subset=['Product_code','number_of_reveiws'], keep='first')
#print(df['Product_code'].value_counts())
f = lambda x: x.split(' ')[1] if x.startswith('(Renewed)') or x.startswith('2020') or x.startswith('Newest')  else x.split(' ')[0]
df['Brand'] = df['Product_name'].map(f)
df['number_of_reveiws']=df['number_of_reveiws'].astype(int)
df['Brand']=df['Brand'].apply(lambda x : x.upper())
df['Price']=df['Price'].str.replace(',', '')
df['Price']=df['Price'].str.replace('₹', '')
df['Price']=df['Price'].astype(int)
#df['Renewed'] = list(map(lambda x: x.startswith('(Renewed)'), df['Product_name'])) 
#df['Brand'].value_counts()

df.to_csv('Products.csv')
print(len(df))
df.head()


206


,Product_code,Product_name,Rating,number_of_reveiws,Price,Images,Brand
0,B097RKZYGS,"ASUS VivoBook 14 (2020), 14-inch (35.56 cms) F...",3.9,156,31990,https://m.media-amazon.com/images/I/716bvqzz7P...,ASUS
1,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.9,766,25999,https://m.media-amazon.com/images/I/613WULNEr1...,HP
2,B08HJYFZ14,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",3.5,483,25999,https://m.media-amazon.com/images/I/61bkuSlKF4...,HP
3,B08HJZHTM1,HP 15 Intel Pentium Gold 6405U Processor Entry...,3.4,843,25999,https://m.media-amazon.com/images/I/71FB8qoB78...,HP
4,B0924PSV78,RDP ThinBook 1010 - Intel Celeron Quad Core Pr...,3.4,258,19990,https://m.media-amazon.com/images/I/51C6EHkMQp...,RDP


In [ ]:
df = pd.read_csv('Products.csv')
df.head()

,Unnamed: 0,Product_code,Product_name,Rating,number_of_reveiws,Price,Images,Brand
0,0,B097RKZYGS,"ASUS VivoBook 14 (2020), 14-inch (35.56 cms) F...",3.9,156,31990,https://m.media-amazon.com/images/I/716bvqzz7P...,ASUS
1,1,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.9,766,25999,https://m.media-amazon.com/images/I/613WULNEr1...,HP
2,2,B08HJYFZ14,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",3.5,483,25999,https://m.media-amazon.com/images/I/61bkuSlKF4...,HP
3,3,B08HJZHTM1,HP 15 Intel Pentium Gold 6405U Processor Entry...,3.4,843,25999,https://m.media-amazon.com/images/I/71FB8qoB78...,HP
4,4,B0924PSV78,RDP ThinBook 1010 - Intel Celeron Quad Core Pr...,3.4,258,19990,https://m.media-amazon.com/images/I/51C6EHkMQp...,RDP


In [ ]:
#test
"""
df['Price']=df['Price'].str.replace(',', '')
df['Price']=df['Price'].str.replace('₹', '')
data=df
def get_data_frame(brand,ub,lb,sort):
    table = pd.DataFrame(columns = data.columns)
    if ub==-1:
        table=data.loc[data['Price']>100000]
    elif lb>0 and ub>0:
        table=data.loc[(data['Price']>lb)&(data['Price']<=ub)]
    else:
        table=data
    if brand!='Any':
        table=table.loc[table['Brand']==brand]
    if sort=='Price_A':
        table=table.nsmallest(5,['Price'])
    elif sort=='Price_D':
        table=table.nlargest(5,['Price'])
    else:
        table=table.nlargest(5,[sort])
    return table

get_data_frame('Any',35000,20000,'Price_D')
display=df[['Product_code','Brand','Product_name','Price','Rating','number of reveiws','Images']]
display.to_csv('Products.csv')
all_data=[]
for i in range(display.shape[0]):
  all_data.append(dict(display.iloc[i]))
print(all_data[1])
#display.head()
  """

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/DibyaSadhukhan/Amazon_Review_Analysis/main/Data/Top_products.csv')
data.head()

,Unnamed: 0,Product_code,Product_name,Rating,number_of_reveiws,Price,Images,Brand
0,36,B08T6THSMQ,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",4.2,1679,40999,https://m.media-amazon.com/images/I/71SMr4ZGf1...,HP
1,35,B08T6SJ215,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",3.9,1673,36999,https://m.media-amazon.com/images/I/71SMr4ZGf1...,HP
2,136,B08XNYN5QV,HP 15 (2021) Thin & Light 11th Gen Core i5 Lap...,4.2,1528,43290,https://m.media-amazon.com/images/I/71uOkXZyte...,HP
3,57,B095JZLDSF,HP 14 (2021) Thin & Light 11th Gen Core i3 Lap...,3.4,1312,41990,https://m.media-amazon.com/images/I/71CiYVzgJs...,HP
4,30,B08DDYYZ3G,"AVITA Essential NE14A2INC433-CR 14"" (35.56cms)...",3.9,1155,37990,https://m.media-amazon.com/images/I/61NsZ6UrfP...,AVITA


In [ ]:
data['Product_name']=data['Product_name'].apply(lambda x : ' '.join(x.split(' ')[:8])+'...')


In [ ]:
data.to_csv('Top_products.csv')

#Block to extract the laptop details from the details file


In [3]:
df=pd.read_csv('/content/gdrive/MyDrive/Laptop_Reveiw/Data/Master_reveiw_2021-11-06.csv')
df=df.drop([df.columns[0],df.columns[-3]], axis=1)
df['Brand']=None
df['Screen_size']=None
df['RAM']=None
df['Processor']=None
df['Hard_drive']=""
df['Type']=None
df['all_det']=None
df.head()

,Product_code,Product_name,Rating,number of reveiws,Price,Images,Link,Reveiw Link,details_file,Brand,Screen_size,RAM,Processor,Hard_drive,Type,all_det
0,B08CKY5XX3,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",4.1 out of 5 stars,659,"₹27,639",https://m.media-amazon.com/images/I/71S8U9VzLT...,https://www.amazon.in/ASUS-VivoBook-Integrated...,https://www.amazon.in/ASUS-VivoBook-Integrated...,/content/gdrive/MyDrive/Amazon_product_Details...,None,None,None,None,,None,None
1,B095YC11CF,HP 15 10th Gen Intel Core i3 Thin and Light 15...,3.8 out of 5 stars,1511,"₹40,990",https://m.media-amazon.com/images/I/81Ne5qKmE8...,https://www.amazon.in/HP-inches-Laptop-Windows...,https://www.amazon.in/HP-inches-Laptop-Windows...,/content/gdrive/MyDrive/Amazon_product_Details...,None,None,None,None,,None,None
2,B08X4N3DW2,"Dell Inspiron 3502 15.6"" (39.62 cms) HD Displa...",4.2 out of 5 stars,276,"₹44,125",https://m.media-amazon.com/images/I/51jPUwqQTF...,https://www.amazon.in/Dell-Inspiron-Integrated...,https://www.amazon.in/Dell-Inspiron-Integrated...,/content/gdrive/MyDrive/Amazon_product_Details...,None,None,None,None,,None,None
3,B09G9ZPV38,Lenovo IdeaPad Slim 1 Intel Celeron N4020 11.6...,3.8 out of 5 stars,92,"₹25,990",https://m.media-amazon.com/images/I/61km1VkaCs...,https://www.amazon.in/Lenovo-IdeaPad-Celeron-P...,https://www.amazon.in/Lenovo-IdeaPad-Celeron-P...,/content/gdrive/MyDrive/Amazon_product_Details...,None,None,None,None,,None,None
4,B0928NL6F3,HP 14 11th Gen Intel Core i5 Processor 14 inch...,4.1 out of 5 stars,417,"₹57,990",https://m.media-amazon.com/images/I/71bJqS8ZLT...,https://www.amazon.in/HP-Processor-Windows-Nat...,https://www.amazon.in/HP-Processor-Windows-Nat...,/content/gdrive/MyDrive/Amazon_product_Details...,None,None,None,None,,None,None


In [4]:
details_file=[]
for i in range(len(df)):
  details_file.append(df.iat[i,-8])
#print(details_file)
i=0
for link in details_file:
  try:
    f = open(link,"r")
    a=f.read().split('||')
    for det in a:
      th=det.split(':')[0]
      td=det.split(':')[1:]
      if th=='Brand':
        df.iat[i,-7]=''.join(td).upper()
      elif th=='RAM Size':
        df.iat[i,-5]=''.join(td).upper()
      elif th=='Standing screen display size':
        df.iat[i,-6]=''.join(td).split(' ')[0]
      elif th=='Processor Type':
        df.iat[i,-4]=''.join(td)
      elif th=='Hard Disk Description':
        df.iat[i,-3]=''.join(td)
      elif th=='Device Type':
        df.iat[i,-2]=''.join(td)
    f.close()
    i+=1
  except:
    i+=1
    #print(link)
    print(i)
df.head(6)

,Product_code,Product_name,Rating,number of reveiws,Price,Images,Link,Reveiw Link,details_file,Brand,Screen_size,RAM,Processor,Hard_drive,Type,all_det
0,B08CKY5XX3,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",4.1 out of 5 stars,659,"₹27,639",https://m.media-amazon.com/images/I/71S8U9VzLT...,https://www.amazon.in/ASUS-VivoBook-Integrated...,https://www.amazon.in/ASUS-VivoBook-Integrated...,/content/gdrive/MyDrive/Amazon_product_Details...,‎ASUS,‎15.6,‎4 GB,‎Celeron N4020,‎Solid State Drive,‎Ultrabook,None
1,B095YC11CF,HP 15 10th Gen Intel Core i3 Thin and Light 15...,3.8 out of 5 stars,1511,"₹40,990",https://m.media-amazon.com/images/I/81Ne5qKmE8...,https://www.amazon.in/HP-inches-Laptop-Windows...,https://www.amazon.in/HP-inches-Laptop-Windows...,/content/gdrive/MyDrive/Amazon_product_Details...,‎HP,‎15.6,‎8 GB,‎Core i3,‎Mechanical Hard Drive,None,None
2,B08X4N3DW2,"Dell Inspiron 3502 15.6"" (39.62 cms) HD Displa...",4.2 out of 5 stars,276,"₹44,125",https://m.media-amazon.com/images/I/51jPUwqQTF...,https://www.amazon.in/Dell-Inspiron-Integrated...,https://www.amazon.in/Dell-Inspiron-Integrated...,/content/gdrive/MyDrive/Amazon_product_Details...,‎DELL,‎15.6,‎4 GB,‎Pentium Other,‎Solid State Drive,None,None
3,B09G9ZPV38,Lenovo IdeaPad Slim 1 Intel Celeron N4020 11.6...,3.8 out of 5 stars,92,"₹25,990",https://m.media-amazon.com/images/I/61km1VkaCs...,https://www.amazon.in/Lenovo-IdeaPad-Celeron-P...,https://www.amazon.in/Lenovo-IdeaPad-Celeron-P...,/content/gdrive/MyDrive/Amazon_product_Details...,‎LENOVO,‎11.6,‎4 GB,‎Celeron N4020,‎SSD,‎Notebook,None
4,B0928NL6F3,HP 14 11th Gen Intel Core i5 Processor 14 inch...,4.1 out of 5 stars,417,"₹57,990",https://m.media-amazon.com/images/I/71bJqS8ZLT...,https://www.amazon.in/HP-Processor-Windows-Nat...,https://www.amazon.in/HP-Processor-Windows-Nat...,/content/gdrive/MyDrive/Amazon_product_Details...,‎HP,‎14,‎8 GB,‎Core i5,‎SSD,None,None
5,B08C5W6PL2,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.8 out of 5 stars,862,"₹26,750",https://m.media-amazon.com/images/I/613WULNEr1...,https://www.amazon.in/HP-Chromebook-14a-na0003...,https://www.amazon.in/HP-Chromebook-14a-na0003...,/content/gdrive/MyDrive/Amazon_product_Details...,‎HP,‎14,‎4 GB,‎Celeron,‎Solid State Drive,None,None


In [9]:
#df['Type'].value_counts()
df.loc[df['Type']=='‎Gaming laptop']

,Product_code,Product_name,Rating,number of reveiws,Price,Images,Link,Reveiw Link,details_file,Brand,Screen_size,RAM,Processor,Hard_drive,Type,all_det
21,B098P62W63,"ASUS TUF Gaming F15, 15.6-inch (39.62 cms) FHD...",4.4 out of 5 stars,1169,"₹58,999",https://m.media-amazon.com/images/I/914o5xV1+8...,https://www.amazon.in/ASUS-15-6-inch-i5-10300H...,https://www.amazon.in/ASUS-15-6-inch-i5-10300H...,/content/gdrive/MyDrive/Amazon_product_Details...,‎ASUS,‎39.62,‎8 GB,‎Core i5,‎SSD,‎Gaming laptop,None
39,B09H7G428M,Lenovo IdeaPad Gaming 3 AMD Ryzen 5 4600H 15.6...,4.4 out of 5 stars,330,"₹55,289",https://m.media-amazon.com/images/I/61OhtnNT6-...,https://www.amazon.in/Lenovo-IdeaPad-82EY00UAI...,https://www.amazon.in/Lenovo-IdeaPad-82EY00UAI...,/content/gdrive/MyDrive/Amazon_product_Details...,‎LENOVO,‎15.6,‎8 GB,None,‎Solid State Hard Drive,‎Gaming laptop,None
66,B09CCNMXB1,ASUS TUF Dash F15 (2021) 15.6-inch (39.62 cms)...,3.8 out of 5 stars,12,"₹93,990",https://m.media-amazon.com/images/I/81stVUZ+Wu...,https://www.amazon.in/ASUS-Dash-F15-15-6-inch-...,https://www.amazon.in/ASUS-Dash-F15-15-6-inch-...,/content/gdrive/MyDrive/Amazon_product_Details...,‎ASUS,‎39.62,‎4 GB,‎Core i7,‎SSD,‎Gaming laptop,None
76,B094XL5V92,Lenovo Legion Y540 9th Gen Intel Core i5-9300H...,4.2 out of 5 stars,170,"₹62,990",https://m.media-amazon.com/images/I/7110pLjNCi...,https://www.amazon.in/Lenovo-i5-9300HF-39-63cm...,https://www.amazon.in/Lenovo-i5-9300HF-39-63cm...,/content/gdrive/MyDrive/Amazon_product_Details...,‎LENOVO,‎15.6,‎8 GB,‎Core i5,‎SSHD,‎Gaming laptop,None
166,B08JLR7LPQ,Lenovo Legion 5 AMD Ryzen 5 4600H 15.6 inch (3...,4.5 out of 5 stars,875,"₹63,990",https://m.media-amazon.com/images/I/61ItfhQmaF...,https://www.amazon.in/Lenovo-Windows-Graphics-...,https://www.amazon.in/Lenovo-Windows-Graphics-...,/content/gdrive/MyDrive/Amazon_product_Details...,‎LENOVO,‎15.6,‎8 GB,‎Ryzen 5 4600H,‎Hybrid Drive,‎Gaming laptop,None
170,B097TY6PTN,Lenovo Legion 5 10th Gen Intel Core i7-10750H ...,4.0 out of 5 stars,24,"₹81,990",https://m.media-amazon.com/images/I/61p3lA4N3u...,https://www.amazon.in/Lenovo-i7-10750H-39-63cm...,https://www.amazon.in/Lenovo-i7-10750H-39-63cm...,/content/gdrive/MyDrive/Amazon_product_Details...,‎LENOVO,‎15.6,‎16 GB,‎Core i7,‎Solid State Drive,‎Gaming laptop,None
171,B09DMYZW5Q,"Lenovo Legion 5 11th Gen Intel Core i7 15.6""(3...",4.4 out of 5 stars,165,"₹1,04,990",https://m.media-amazon.com/images/I/61Er61SxBh...,https://www.amazon.in/Lenovo-39-62cm-Refresh-K...,https://www.amazon.in/Lenovo-39-62cm-Refresh-K...,/content/gdrive/MyDrive/Amazon_product_Details...,‎LENOVO,‎15.6,‎16 GB,‎Core i7,‎SSD,‎Gaming laptop,None
180,B09CD4FRNX,ASUS TUF Dash F15 (2021) 15.6-inch (39.62 cms)...,2.9 out of 5 stars,9,"₹83,990",https://m.media-amazon.com/images/I/81mLxU0CcA...,https://www.amazon.in/ASUS-Dash-F15-15-6-inch-...,https://www.amazon.in/ASUS-Dash-F15-15-6-inch-...,/content/gdrive/MyDrive/Amazon_product_Details...,‎ASUS,‎39.62,‎4 GB,‎Core i5,‎SSD,‎Gaming laptop,None
